Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Необходимо:

1. Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
2. Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
3. Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
4. Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).

In [68]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px


# чтение данных
data_parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing', header=0)
data_company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company', header=0)

def clear_pars(str):
    '''Функция очистки строк от служебных символов'''
    simblist = ['_', '-', '~', '\\', '/']
    for simb in str:
        if simb in simblist:
            str = str.replace(simb, '')
    return str

# очищаем колонку 'producer_color' и 'producer_id' от служебных символов
data_parsing['producer_color'] = data_parsing['producer_color'].apply(clear_pars)
data_parsing['producer_id'] = data_parsing['producer_id'].apply(clear_pars)

# display(data_parsing, data_company)

# объединяем таблицы по признаку 'producer_id'
data_parsing = data_parsing.merge(right=data_company, how='left', left_on='producer_id', right_on='item_id')

# создаем колонку с разницей цены в %
data_parsing['dif_price'] = data_parsing['price'] / data_parsing['current price'] * 100

# удаляем ненужные столбцы
data_parsing = data_parsing.drop(['brand_y','Category_y', 'color_id', 'item_id'], axis=1)
display(data_parsing)

#строим боксплот по категориям и брендам (3 бренда, 2 категории)
fig = px.box(
    data_frame=data_parsing, #DataFrame
    y='brand_x', 
    x='dif_price', 
    height=400, #высота
    width=800, #ширина
    labels={'brand_x':'Брэнд', 'dif_price': 'Отклонение цены, %'},
    title='Отклонение цены по брендам',
    color='brand_x'
       
)
#отображаем график
fig.show()

fig = px.box(
    data_frame=data_parsing, #DataFrame
    y='Category_x', 
    x='dif_price', 
    height=400, #высота
    width=800, #ширина
    labels={'Category_x':'Категории', 'dif_price': 'Отклонение цены, %'},
    title='Отклонение цены по категориям',
    color='Category_x'
       
)
#отображаем график
fig.show()

,brand_x,Category_x,producer_id,producer_color,price,current price,dif_price
0,Valentino,Shoes,aaaaa111111,black,167,247,67.611336
1,Valentino,Shoes,aaaaa111112,black,188,161,116.770186
2,Valentino,Shoes,aaaaa111113,black,184,234,78.632479
3,Valentino,Shoes,aaaaa111114,black,196,167,117.365269
4,Valentino,Shoes,aaaaa111115,black,250,153,163.398693
...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,165,99.393939
71,Stone Island,Bags,sssss111132,red,158,196,80.612245
72,Stone Island,Bags,sssss111133,red,194,236,82.203390
73,Stone Island,Bags,sssss111134,red,256,222,115.315315


Выводы по графикам:
- выбросы на графиках не наблюдаются, но видно смещение усов в правую сторону.
Пройдем по брендам методами z-отклонений.

In [79]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

brand_list = list(data_parsing['brand_x'].value_counts().index)

# запустим цикл, в котором с каждой итерацией по бренду будем получать выбросы в признаке 'price_difference'
for brand_name in brand_list:

    mask = data_parsing['brand_x'] == brand_name
    outliers, cleaned = outliers_z_score(data_parsing[mask], 'dif_price')
    print(f'Число выбросов по методу z-отклонения в бренде {brand_name}: {outliers.shape[0]}')
    print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения в бренде Valentino: 0
Результирующее число записей: 25
Число выбросов по методу z-отклонения в бренде Brioni: 0
Результирующее число записей: 25
Число выбросов по методу z-отклонения в бренде Stone Island: 0
Результирующее число записей: 25


In [80]:
for brand_name in brand_list:

    mask = data_parsing['brand_x'] == brand_name
    outliers, cleaned = outliers_z_score_mod(data_parsing[mask], 'dif_price', left=2, right=2)
    print(f'Число выбросов по методу z-отклонения в бренде {brand_name}: {outliers.shape[0]}')
    print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения в бренде Valentino: 1
Результирующее число записей: 24
Число выбросов по методу z-отклонения в бренде Brioni: 2
Результирующее число записей: 23
Число выбросов по методу z-отклонения в бренде Stone Island: 2
Результирующее число записей: 23
